In [1]:
import numpy as np
import math
import plotly.graph_objects as go

# =============================
# 0) Synthetic underlier setup
# =============================
S0 = 100.0
r = 0.0
T = 30 / 365        # 1 month
F = S0 * math.exp(r * T)

K = np.arange(60, 141, 5).astype(float)
x = np.log(K / F)

# =============================
# 1) Black–Scholes helpers
# =============================
def norm_cdf(z):
    return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))

def bs_call_price(S, K, r, T, sigma):
    if T <= 0 or sigma <= 0:
        return max(S - K, 0.0)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

# ==========================================
# 2) Breeden–Litzenberger implied PDF
# ==========================================
def bl_pdf_from_iv(iv):
    call_mid = np.array([
        bs_call_price(S0, k, r, T, s)
        for k, s in zip(K, iv)
    ])

    dK = K[1] - K[0]
    C2 = np.full_like(call_mid, np.nan)
    C2[1:-1] = (call_mid[2:] - 2 * call_mid[1:-1] + call_mid[:-2]) / dK**2

    pdf_raw = np.full_like(call_mid, np.nan)
    pdf_raw[1:-1] = math.exp(r * T) * C2[1:-1]

    # Clip numerical noise
    pdf = np.maximum(pdf_raw, 0.0)

    # Normalize
    mass = np.nansum(pdf) * dK
    pdf_norm = pdf / mass if mass > 0 else pdf

    return pdf_norm

# ==========================================
# 3) Base IV surface
# ==========================================
base = 0.45
skew = -0.35
smile = 0.70
right_wing = 1.20

iv_base = (
    base
    + skew * x
    + smile * x**2
    + right_wing * np.maximum(x, 0)**3
)
iv_base = np.clip(iv_base, 0.10, 3.00)

# ==========================================
# 4) Scenario 1: left wing up
# ==========================================
left_wing_boost = 2.0
iv_left = iv_base + left_wing_boost * np.maximum(-x, 0)**2
iv_left = np.clip(iv_left, 0.10, 3.00)

# ==========================================
# 5) Scenario 2: ATM up
# ==========================================
atm_bump = 0.15
width = 0.18
iv_atm = iv_base + atm_bump * np.exp(-(x / width)**2)
iv_atm = np.clip(iv_atm, 0.10, 3.00)

# ==========================================
# 6) PDFs
# ==========================================
pdf_base = bl_pdf_from_iv(iv_base)
pdf_left = bl_pdf_from_iv(iv_left)
pdf_atm  = bl_pdf_from_iv(iv_atm)

# ==========================================
# 7) Plotly: IV Smile comparison
# ==========================================
fig_iv = go.Figure()

fig_iv.add_trace(go.Scatter(
    x=K, y=iv_base,
    mode="lines+markers",
    name="Base"
))

fig_iv.add_trace(go.Scatter(
    x=K, y=iv_left,
    mode="lines+markers",
    name="Scenario 1: Left wing up"
))

fig_iv.add_trace(go.Scatter(
    x=K, y=iv_atm,
    mode="lines+markers",
    name="Scenario 2: ATM up"
))

fig_iv.update_layout(
    title="IV Smile Comparison (Same Maturity)",
    xaxis_title="Strike K",
    yaxis_title="Implied Volatility",
    template="plotly_white"
)

fig_iv.show()

# ==========================================
# 8) Plotly: BL implied PDF comparison
# ==========================================
fig_pdf = go.Figure()

fig_pdf.add_trace(go.Scatter(
    x=K, y=pdf_base,
    mode="lines+markers",
    name="Base"
))

fig_pdf.add_trace(go.Scatter(
    x=K, y=pdf_left,
    mode="lines+markers",
    name="Scenario 1: Left wing up"
))

fig_pdf.add_trace(go.Scatter(
    x=K, y=pdf_atm,
    mode="lines+markers",
    name="Scenario 2: ATM up"
))

fig_pdf.update_layout(
    title="Implied Risk-Neutral PDF (Breeden–Litzenberger)",
    xaxis_title="Price at Expiry (mapped to K)",
    yaxis_title="Probability Density (normalized)",
    template="plotly_white"
)

fig_pdf.show()


In [2]:
import numpy as np
import math
import plotly.graph_objects as go
from scipy.stats import norm

# =============================
# 0) Market setup
# =============================
S0 = 100.0
r = 0.0
T = 30 / 365
F = S0 * math.exp(r * T)

K = np.arange(60, 141, 5).astype(float)
x = np.log(K / F)

# =============================
# 1) Black–Scholes helpers
# =============================
def bs_d1(S, K, r, T, sigma):
    return (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def bs_call_price(S, K, r, T, sigma):
    d1 = bs_d1(S, K, r, T, sigma)
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

def bs_call_delta(S, K, r, T, sigma):
    d1 = bs_d1(S, K, r, T, sigma)
    return norm.cdf(d1)

# =============================
# 2) Breeden–Litzenberger PDF
# =============================
def bl_pdf_from_iv(iv):
    call_mid = np.array([
        bs_call_price(S0, k, r, T, s)
        for k, s in zip(K, iv)
    ])

    dK = K[1] - K[0]
    C2 = np.zeros_like(call_mid)
    C2[1:-1] = (call_mid[2:] - 2*call_mid[1:-1] + call_mid[:-2]) / dK**2

    pdf = np.maximum(C2, 0.0)
    pdf /= np.sum(pdf) * dK
    return pdf

# =============================
# 3) Base IV surface
# =============================
base = 0.45
skew = -0.35
smile = 0.70
right_wing = 1.20

iv = base + skew*x + smile*x**2 + right_wing*np.maximum(x, 0)**3
iv = np.clip(iv, 0.10, 3.00)

# =============================
# 4) Compute PDF
# =============================
pdf = bl_pdf_from_iv(iv)

# =============================
# 5) Desk metrics: ATM, 25Δ, 10Δ
# =============================
delta_call = np.array([
    bs_call_delta(S0, k, r, T, s)
    for k, s in zip(K, iv)
])

def interp_iv(target_delta):
    return np.interp(target_delta, delta_call[::-1], iv[::-1])

iv_atm = np.interp(0.5, delta_call[::-1], iv[::-1])
iv_25c = interp_iv(0.25)
iv_25p = interp_iv(0.75)
iv_10c = interp_iv(0.10)
iv_10p = interp_iv(0.90)

rr_25 = iv_25c - iv_25p
rr_10 = iv_10c - iv_10p

# =============================
# 6) Plot 1: IV Smile
# =============================
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=K, y=iv, mode="lines+markers", name="IV Smile"))
fig1.update_layout(
    title="IV Smile (Strike Space)",
    xaxis_title="Strike K",
    yaxis_title="Implied Volatility",
    template="plotly_white"
)
fig1.show()

# =============================
# 7) Plot 2: Implied PDF
# =============================
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=K, y=pdf, mode="lines+markers", name="PDF"))
fig2.update_layout(
    title="Implied Risk-Neutral PDF (BL)",
    xaxis_title="Price at Expiry",
    yaxis_title="Probability Density",
    template="plotly_white"
)
fig2.show()

# =============================
# 8) Plot 3: Desk View (Δ-space)
# =============================
fig3 = go.Figure()

fig3.add_bar(name="ATM IV", x=["ATM"], y=[iv_atm])
fig3.add_bar(name="25Δ RR", x=["25Δ RR"], y=[rr_25])
fig3.add_bar(name="10Δ RR", x=["10Δ RR"], y=[rr_10])

fig3.update_layout(
    title="Desk Metrics View",
    yaxis_title="Vol / Vol Spread",
    template="plotly_white"
)
fig3.show()


In [3]:
import numpy as np
import math
import plotly.graph_objects as go
from scipy.stats import norm

# =============================
# 0) Market setup
# =============================
S0 = 100.0
r = 0.0
T = 30 / 365
F = S0 * math.exp(r * T)

K = np.arange(60, 141, 5).astype(float)
x = np.log(K / F)
dK = K[1] - K[0]

# =============================
# 1) Black–Scholes helpers
# =============================
def bs_d1(S, K, r, T, sigma):
    return (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def bs_call_price(S, K, r, T, sigma):
    d1 = bs_d1(S, K, r, T, sigma)
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

# =============================
# 2) Base IV surface (synthetic)
# =============================
base = 0.45
skew = -0.35
smile = 0.70
right_wing = 1.20

iv = base + skew*x + smile*x**2 + right_wing*np.maximum(x, 0)**3
iv = np.clip(iv, 0.10, 3.00)

# =============================
# 3) Call prices
# =============================
C = np.array([
    bs_call_price(S0, k, r, T, s)
    for k, s in zip(K, iv)
])

# =============================
# 4) CDF: -dC/dK  (digital / tail prob)
# =============================
dC_dK = np.zeros_like(C)
dC_dK[1:-1] = (C[2:] - C[:-2]) / (2 * dK)

CDF = -dC_dK                 # ≈ e^{-rT} P(S_T > K)
CDF = np.clip(CDF, 0.0, 1.0)

# =============================
# 5) PDF: d²C/dK²  (BL density)
# =============================
d2C_dK2 = np.zeros_like(C)
d2C_dK2[1:-1] = (C[2:] - 2*C[1:-1] + C[:-2]) / (dK**2)

PDF = np.maximum(d2C_dK2, 0.0)
PDF /= np.sum(PDF) * dK      # normalize

# =============================
# 6) Plot 1: IV Smile
# =============================
fig_iv = go.Figure()
fig_iv.add_trace(go.Scatter(
    x=K, y=iv,
    mode="lines+markers",
    name="IV Smile"
))
fig_iv.update_layout(
    title="IV Smile (Pricing Input)",
    xaxis_title="Strike K",
    yaxis_title="Implied Volatility",
    template="plotly_white"
)
fig_iv.show()

# =============================
# 7) Plot 2: CDF  (-dC/dK)
# =============================
fig_cdf = go.Figure()
fig_cdf.add_trace(go.Scatter(
    x=K, y=CDF,
    mode="lines+markers",
    name="CDF: P(S_T > K)"
))
fig_cdf.update_layout(
    title="Implied CDF  ( - ∂C / ∂K )",
    xaxis_title="Strike K",
    yaxis_title="Tail Probability",
    template="plotly_white"
)
fig_cdf.show()

# =============================
# 8) Plot 3: PDF  (BL)
# =============================
fig_pdf = go.Figure()
fig_pdf.add_trace(go.Scatter(
    x=K, y=PDF,
    mode="lines+markers",
    name="PDF (BL density)"
))
fig_pdf.update_layout(
    title="Implied PDF  ( ∂²C / ∂K² )",
    xaxis_title="Price at Expiry (mapped to K)",
    yaxis_title="Probability Density",
    template="plotly_white"
)
fig_pdf.show()


In [4]:
import numpy as np
import math
import plotly.graph_objects as go
from scipy.stats import norm

# =============================
# 0) Market setup
# =============================
S0 = 100.0
r = 0.0
T = 30 / 365
F = S0 * math.exp(r * T)

K = np.arange(60, 141, 5).astype(float)
x = np.log(K / F)
dK = K[1] - K[0]

# =============================
# 1) Black–Scholes helpers
# =============================
def bs_d1(S, K, r, T, sigma):
    return (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def bs_call_price(S, K, r, T, sigma):
    d1 = bs_d1(S, K, r, T, sigma)
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

# =============================
# 2) BL CDF / PDF from IV
# =============================
def compute_cdf_pdf(iv):
    C = np.array([
        bs_call_price(S0, k, r, T, s)
        for k, s in zip(K, iv)
    ])

    # CDF: -dC/dK
    dC_dK = np.zeros_like(C)
    dC_dK[1:-1] = (C[2:] - C[:-2]) / (2 * dK)
    CDF = np.clip(-dC_dK, 0.0, 1.0)

    # PDF: d2C/dK2
    d2C_dK2 = np.zeros_like(C)
    d2C_dK2[1:-1] = (C[2:] - 2*C[1:-1] + C[:-2]) / (dK**2)
    PDF = np.maximum(d2C_dK2, 0.0)
    PDF /= np.sum(PDF) * dK

    return CDF, PDF

# =============================
# 3) Base IV surface
# =============================
base = 0.45
skew = -0.35
smile = 0.70
right_wing = 1.20

iv_base = (
    base
    + skew * x
    + smile * x**2
    + right_wing * np.maximum(x, 0)**3
)
iv_base = np.clip(iv_base, 0.10, 3.00)

# =============================
# 4) Scenario A: ATM up (body only)
# =============================
atm_bump = 0.15
width = 0.18
iv_atm = iv_base + atm_bump * np.exp(-(x / width)**2)
iv_atm = np.clip(iv_atm, 0.10, 3.00)

# =============================
# 5) Scenario B: Left wing up
# =============================
left_wing_boost = 2.0
iv_left = iv_base + left_wing_boost * np.maximum(-x, 0)**2
iv_left = np.clip(iv_left, 0.10, 3.00)

# =============================
# 6) Compute CDF / PDF
# =============================
cdf_base, pdf_base = compute_cdf_pdf(iv_base)
cdf_atm,  pdf_atm  = compute_cdf_pdf(iv_atm)
cdf_left, pdf_left = compute_cdf_pdf(iv_left)

# =============================
# 7) Plot 1: IV Smile comparison
# =============================
fig_iv = go.Figure()
fig_iv.add_trace(go.Scatter(x=K, y=iv_base, mode="lines+markers", name="Base"))
fig_iv.add_trace(go.Scatter(x=K, y=iv_atm,  mode="lines+markers", name="ATM up"))
fig_iv.add_trace(go.Scatter(x=K, y=iv_left, mode="lines+markers", name="Left wing up"))

fig_iv.update_layout(
    title="IV Smile Comparison",
    xaxis_title="Strike K",
    yaxis_title="Implied Volatility",
    template="plotly_white"
)
fig_iv.show()

# =============================
# 8) Plot 2: CDF comparison
# =============================
fig_cdf = go.Figure()
fig_cdf.add_trace(go.Scatter(x=K, y=cdf_base, mode="lines+markers", name="Base"))
fig_cdf.add_trace(go.Scatter(x=K, y=cdf_atm,  mode="lines+markers", name="ATM up"))
fig_cdf.add_trace(go.Scatter(x=K, y=cdf_left, mode="lines+markers", name="Left wing up"))

fig_cdf.update_layout(
    title="Implied CDF  ( P(S_T > K) )",
    xaxis_title="Strike K",
    yaxis_title="Tail Probability",
    template="plotly_white"
)
fig_cdf.show()

# =============================
# 9) Plot 3: PDF comparison
# =============================
fig_pdf = go.Figure()
fig_pdf.add_trace(go.Scatter(x=K, y=pdf_base, mode="lines+markers", name="Base"))
fig_pdf.add_trace(go.Scatter(x=K, y=pdf_atm,  mode="lines+markers", name="ATM up"))
fig_pdf.add_trace(go.Scatter(x=K, y=pdf_left, mode="lines+markers", name="Left wing up"))

fig_pdf.update_layout(
    title="Implied PDF  (BL Density)",
    xaxis_title="Price at Expiry (mapped to K)",
    yaxis_title="Probability Density",
    template="plotly_white"
)
fig_pdf.show()
